In [114]:
def find_ntt_prime(n,bits):

    qs =[];

    k = 1;

    while True:
        q = 2*n*k+1;

        if q > 2^bits:
            break;

        if q in Primes():
            qs.append(q);

        k += 1;

    return qs;

In [115]:
def find_generator(q):

    Zq = IntegerModRing(q);

    gs = range(1,q);

    for x in list(factor(q-1)):
        p = x[0];

        t = [];

        for g in gs:
            if Zq(g)^((q-1)/p) != 1:
                t.append(Zq(g));

        gs = t;

    return gs;

In [116]:
def find_w(q):

    k = Integer((q-1)/(2*n));

    ws = [g^k for g in find_generator(q)];

    return sorted(list(set(ws)));

In [130]:
class NTT:
    def __init__(self,n,q,w):
        self.n = n;

        Zq = IntegerModRing(q);

        self.two_inv = Zq(2)^(q-2);
        self.level = Integer(log(n,2));

        self.zetas = [];

        tree = zero_matrix(ZZ,self.level+1,1 << self.level);
        tree[0,0] = self.n;

        for l in range(self.level):
            for i in range(1 << l):
                tree[l+1,2*i  ] = tree[l  ,  i] / 2;
                tree[l+1,2*i+1] = tree[l+1,2*i] + self.n;

                self.zetas.append(Zq(w)^tree[l+1,2*i]);

        print(tree)

    def ntt(self, a):
        k = 0;

        step = self.n/2;

        b = a.copy();

        while step >= 1:
            for start in range(0,self.n,(step << 1)):
                zeta = self.zetas[k];
                k += 1;

                for i in range(start,start+step):
                    t = zeta * b[i+step];

                    b[i+step] = Zq(b[i]-t);
                    b[i     ] = Zq(b[i]+t);

            step >>= 1;

        return b;

    def invntt(self, a):
        k = self.n-2;

        step = 1;

        b = a.copy();

        while step <= self.n/2:
            for start in range(0,self.n,(step << 1)):
                zeta = self.zetas[k];
                k -= 1;

                for i in range(start,start+step):
                    t = b[i+step];

                    b[i+step] = Zq(t-b[i]) * zeta;
                    b[i     ] = Zq(t+b[i])

            step <<= 1;

        factor = self.two_inv^self.level;

        b = [b[i]*factor for i in range(n)];

        return b;

In [131]:
def ntt_mul(n,q,a,b):
    Zq = IntegerModRing(q);

    c = [Zq(a[i]*b[i]) for i in range(n)];

    return c;

In [132]:
def poly_mul(n,q,a,b):
    Zq = IntegerModRing(q);
    c = [0]*n;

    for i in range(n):
        for j in range(n-i):
            c[i+j  ] += Zq(a[i]*b[j]);

        for j in range(n-i,n):
            c[i+j-n] -= Zq(a[i]*b[j]);

    return c;

In [141]:
n = 64

In [142]:
qs = find_ntt_prime(n,14);
q = min(qs)
print(q)

257


In [143]:
ws = find_w(q)
w = min(ws)
print(w)

9


In [144]:
ntt_inst = NTT(n,q,w);

[ 64   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[ 32  96   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[ 16  80  48 112   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  8  72  40 104  24  88  56 120   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  

In [124]:
Zq = IntegerModRing(q);

a = [randrange(q) for i in range(n)];
b = [randrange(q) for i in range(n)];

In [125]:
a_ntt = ntt_inst.ntt(a);
b_ntt = ntt_inst.ntt(b);

In [126]:
c_ntt = ntt_mul(n,q,a_ntt,b_ntt);

In [127]:
c_1 = ntt_inst.invntt(c_ntt);

In [128]:
c_2 = poly_mul(n,q,a,b);

In [129]:
c_1 == c_2

True